# LightGBMをカスタムコンテナで利用する手順を学び、SageMakerの動作を理解します

2hを想定

コンテンツ
* カスタムコンテナ(ローカル学習、ローカル推論、学習ジョブ、推論ジョブ）
* SageMaker Training Toolkit導入（コードを外出しにする）：ローカル学習、ローカル推論


## 環境
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

In [1]:
import sys
#Pythonのバージョン情報
sys.version

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) \n[GCC 9.4.0]'

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) \n[GCC 9.4.0]'

In [2]:
# Pythonのバージョン確認 (システムコマンド使用
!python -V

Python 3.8.12


Python 3.8.12

In [3]:
import sagemaker

print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__))

Current SageMaker Python SDK Version =2.109.0


## ライブラリインポート

In [4]:
# This is a sample Python program that trains a simple LightGBM Regression model, and then performs inference.
# This implementation will work on your local computer.
#
# Prerequisites:
#   1. Install required Python packages:
#       pip install boto3 sagemaker pandas scikit-learn
#       pip install 'sagemaker[local]'
#   2. Docker Desktop has to be installed on your computer, and running.
#   3. Open terminal and run the following commands:
#       docker build  -t sagemaker-lightgbm-regression-local container/.
########################################################################################################################


In [5]:
import pandas as pd
from sagemaker.estimator import Estimator
from sagemaker.local import LocalSession
from sagemaker.predictor import csv_serializer
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [6]:
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

# For local training a dummy role will be sufficient
role = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

# 1.データ準備

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [7]:
data = load_boston()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [8]:
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [9]:
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

# 2.カスタムコンテナ作成

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


# ECRでpushしたコンテナのURIを確認

AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを取得し、以下にはりつけます。

In [10]:
#image = 'sagemaker-lightgbm-regression-local'
#image = '805433377179.dkr.ecr.us-west-2.amazonaws.com/sagemaker-lightgbm-regression:latest' # ビルドしたイメージのURI
image = '429775515542.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression:latest'

In [11]:
local_lightgbm = Estimator(
    image,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0})

In [12]:
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

# ローカル学習
ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [14]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location}, logs=True)

Creating umbbb2deq3-algo-1-f0kmt ... 
Creating umbbb2deq3-algo-1-f0kmt ... done
Attaching to umbbb2deq3-algo-1-f0kmt
umbbb2deq3-algo-1-f0kmt | Starting the training.
umbbb2deq3-algo-1-f0kmt | Reading hyperparameters data: /opt/ml/input/config/hyperparameters.json
umbbb2deq3-algo-1-f0kmt | hyperparameters_data: {'boosting_type': 'gbdt', 'objective': 'regression', 'num_leaves': '31', 'learning_rate': '0.05', 'feature_fraction': '0.9', 'bagging_fraction': '0.8', 'bagging_freq': '5', 'verbose': '0'}
umbbb2deq3-algo-1-f0kmt | Found train files: ['/opt/ml/input/data/train/boston_train.csv']
umbbb2deq3-algo-1-f0kmt | Found validation files: ['/opt/ml/input/data/validation/boston_valid.csv']
umbbb2deq3-algo-1-f0kmt | building training and validation datasets
umbbb2deq3-algo-1-f0kmt | Starting training...
umbbb2deq3-algo-1-f0kmt | /opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of 

ローカルモードの学習結果は

Amazon S3
Buckets
sagemaker-us-west-2-805433377179
sagemaker-lightgbm-regression-2022-10-03-06-17-32-054/

に出力されます。


### ローカルサービング

In [15]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=csv_serializer) 

Attaching to f8onwze1xo-algo-1-ixw9g
f8onwze1xo-algo-1-ixw9g | Starting the inference server with 2 workers.
f8onwze1xo-algo-1-ixw9g | [2022-10-04 04:10:34 +0000] [10] [INFO] Starting gunicorn 20.1.0
f8onwze1xo-algo-1-ixw9g | [2022-10-04 04:10:34 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
f8onwze1xo-algo-1-ixw9g | [2022-10-04 04:10:34 +0000] [10] [INFO] Using worker: gevent
f8onwze1xo-algo-1-ixw9g | [2022-10-04 04:10:34 +0000] [12] [INFO] Booting worker with pid: 12
f8onwze1xo-algo-1-ixw9g | [2022-10-04 04:10:34 +0000] [13] [INFO] Booting worker with pid: 13
!f8onwze1xo-algo-1-ixw9g | 172.18.0.1 - - [04/Oct/2022:04:10:37 +0000] "GET /ping HTTP/1.1" 200 1 "-" "python-urllib3/1.26.8"


In [16]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print(predicted)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


f8onwze1xo-algo-1-ixw9g | Invoked with 102 records
19.95642073217597
27.844891841022335
23.747437427003455
21.961517177305176
33.70952263893306
16.546899933876215
20.7577247308279
21.58941351302627
28.44096446328559
21.573610198594977
16.520022349295115
18.56239893242527
33.70952263893306
21.66404760045202
18.839854556333133
20.524517944865078
23.512192914502315
19.720552829648888
14.831841119971708
25.48273874904075
24.232639474441545
21.624005932843115
24.961489794296718
31.737194191676068
21.634052928440624
28.40721160777621
21.408363849719503
14.831841119971708
22.218594550645975
21.174456098551236
21.78791955089051
14.831841119971708
29.996695633096042
22.44097524661187
33.83316205414468
26.41403196992683
33.70952263893306
17.366188662166092
27.56686070285819
30.785697489113854
19.36938873496206
20.70626548555591
17.759853567831996
27.888269821752413
20.521395163186774
14.831841119971708
24.776417537973362
24.965857100129327
19.649289821764185
21.026797620813866
33.70952263893306


# 学習ジョブを発行

In [17]:
from sagemaker import get_execution_role

role = get_execution_role()

In [18]:
est_lightgbm = Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0})

In [20]:
import sagemaker
bucket_name = '<bucket_name>' # input your bucket name
bucket_name = 'demo-lgbm-container'

train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

In [21]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3}, logs=True)

2022-10-04 04:15:55 Starting - Starting the training job...
2022-10-04 04:16:21 Starting - Preparing the instances for trainingProfilerReport-1664856955: InProgress
.........
2022-10-04 04:17:44 Downloading - Downloading input data
2022-10-04 04:17:44 Training - Downloading the training image...
2022-10-04 04:18:20 Training - Training image download completed. Training in progress..Starting the training.
Reading hyperparameters data: /opt/ml/input/config/hyperparameters.json
hyperparameters_data: {'bagging_fraction': '0.8', 'bagging_freq': '5', 'boosting_type': 'gbdt', 'feature_fraction': '0.9', 'learning_rate': '0.05', 'num_leaves': '31', 'objective': 'regression', 'verbose': '0'}
Found train files: ['/opt/ml/input/data/train/boston_train.csv']
Found validation files: ['/opt/ml/input/data/validation/boston_valid.csv']
building training and validation datasets
Starting training...
/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argume

## エンドポイントにデプロイ
waitしない

In [32]:
1

1

In [33]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=False) 

In [36]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


19.95642073217597
27.844891841022335
23.747437427003455
21.961517177305176
33.70952263893306
16.546899933876215
20.7577247308279
21.58941351302627
28.44096446328559
21.573610198594977
16.520022349295115
18.56239893242527
33.70952263893306
21.66404760045202
18.839854556333133
20.524517944865078
23.512192914502315
19.720552829648888
14.831841119971708
25.48273874904075
24.232639474441545
21.624005932843115
24.961489794296718
31.737194191676068
21.634052928440624
28.40721160777621
21.408363849719503
14.831841119971708
22.218594550645975
21.174456098551236
21.78791955089051
14.831841119971708
29.996695633096042
22.44097524661187
33.83316205414468
26.41403196992683
33.70952263893306
17.366188662166092
27.56686070285819
30.785697489113854
19.36938873496206
20.70626548555591
17.759853567831996
27.888269821752413
20.521395163186774
14.831841119971708
24.776417537973362
24.965857100129327
19.649289821764185
21.026797620813866
33.70952263893306
22.770867837558004
25.12436361101226
32.04499227317

### 学習スクリプトをローカルに保存して実行
GitHubから実行したい場合も。

SageMaker Training Toolkitが必要

### カスタムコンテナ作成
trainは含めないように注意しましょう。

Dockerfileにて、
・train を /opt/program/train と配置
・カレントディレクトリを /opt/program に設定
・SageMaker Training Toolkit が /opt/conda/bin/train にインストールされる
・train を実行すると、カレントにある /opt/program/train が実行されてしまう。
解決するには、
・カレントディレクトリを 持ち込みのtrainがある場所にしない
・train をそもそもコンテナに入れない（確実）

In [35]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-toolkit

#cd container
cd container_smtrtoolkit ### 変更点

#chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


Login Succeeded
Sending build context to Docker daemon  19.97kB
Step 1/14 : FROM ubuntu:16.04
 ---> b6f507652425
Step 2/14 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> b93522070e55
Step 3/14 : ARG CONDA_DIR=/opt/conda
 ---> Using cache
 ---> 3c36d88c98af
Step 4/14 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> e3fd517f81e5
Step 5/14 : RUN apt-get update &&     apt-get install -y --no-install-recommends         ca-certificates         cmake         build-essential         gcc         g++         git         nginx         wget &&     wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh &&     /bin/bash Miniconda3-latest-Linux-x86_64.sh -f -b -p $CONDA_DIR &&     export PATH="$CONDA_DIR/bin:$PATH" &&     conda config --set always_yes yes --set changeps1 no &&     conda install -q -y numpy scipy scikit-learn pandas flask gevent gunicorn &&     git clone --recursive --branch stable --depth 1 https://github.com/Microsoft/LightGBM && 

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## 学習(ローカル)

In [38]:
#image = '805433377179.dkr.ecr.us-west-2.amazonaws.com/sagemaker-lightgbm-toolkit:latest'
image = '429775515542.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-toolkit:latest'
#image = <input your own image URI>

In [39]:
est_lightgbm3 = Estimator(
    image,
    role,
    instance_count=1,
    #instance_type="ml.m4.2xlarge",
    instance_type="local",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0},
    #source_dir='./practice_src',
    entry_point='./src/train_practice.py'
    #entry_point='./practice_src/train_practice.sh'
    )
est_lightgbm3.fit({'train':train_s3, 'validation': valid_s3}, logs=True)

Creating m27alddto9-algo-1-5me20 ... 
Creating m27alddto9-algo-1-5me20 ... done
Attaching to m27alddto9-algo-1-5me20
m27alddto9-algo-1-5me20 | 2022-10-04 05:48:08,705 botocore.credentials INFO     Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
m27alddto9-algo-1-5me20 | 2022-10-04 05:48:08,931 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
m27alddto9-algo-1-5me20 | 2022-10-04 05:48:08,932 sagemaker-training-toolkit INFO     Failed to parse hyperparameter boosting_type value gbdt to Json.
m27alddto9-algo-1-5me20 | Returning the value itself
m27alddto9-algo-1-5me20 | 2022-10-04 05:48:08,932 sagemaker-training-toolkit INFO     Failed to parse hyperparameter objective value regression to Json.
m27alddto9-algo-1-5me20 | Returning the value itself
m27alddto9-algo-1-5me20 | 2022-10-04 05:48:08,945 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
m27alddto9-algo-1-5me20 | 2022-10-04 05:48:08,949 sa

## デプロイローカル

In [40]:
predictor3 = est_lightgbm3.deploy(1, 'local', serializer=csv_serializer) 

f8onwze1xo-algo-1-ixw9g | 172.18.0.1 - - [04/Oct/2022:05:48:16 +0000] "GET /ping HTTP/1.1" 200 1 "-" "python-urllib3/1.26.8"
!

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 854, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 916, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 859, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp9zd_hh8c/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1


In [41]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor3.predict(payload).decode('utf-8')
print(predicted)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


f8onwze1xo-algo-1-ixw9g | Invoked with 102 records
19.95642073217597
27.844891841022335
23.747437427003455
21.961517177305176
33.70952263893306
16.546899933876215
20.7577247308279
21.58941351302627
28.44096446328559
21.573610198594977
16.520022349295115
18.56239893242527
33.70952263893306
21.66404760045202
18.839854556333133
20.524517944865078
23.512192914502315
19.720552829648888
14.831841119971708
25.48273874904075
24.232639474441545
21.624005932843115
24.961489794296718
31.737194191676068
21.634052928440624
28.40721160777621
21.408363849719503
14.831841119971708
22.218594550645975
21.174456098551236
21.78791955089051
14.831841119971708
29.996695633096042
22.44097524661187
33.83316205414468
26.41403196992683
33.70952263893306
17.366188662166092
27.56686070285819
30.785697489113854
19.36938873496206
20.70626548555591
17.759853567831996
27.888269821752413
20.521395163186774
14.831841119971708
24.776417537973362
24.965857100129327
19.649289821764185
21.026797620813866
33.70952263893306


# 参考


SageMaker-Pytorth training Toolkit
https://github.com/aws/sagemaker-pytorch-training-toolkit/


SageMaker-Pytorch Inference Toolkit

https://github.com/aws/sagemaker-pytorch-inference-toolkit



https://stackoverflow.com/questions/73694705/what-is-the-difference-between-sagemaker-pytorch-training-toolkit-and-sagemaker


## 参考
SageMaker のtrainingジョブを理解する

https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial

# Toolkitを入れず、train からtrain.shを実行

ソースをS3に配置しなればならない


fit()について

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

datasetの指定は、S3のパスか、ローカルモードならfile://　つまりGitHubは不可

### SageMaker Traiing Toolkitについて

https://github.com/aws/sagemaker-training-toolkit/blob/master/README.md

inference toolkitもある。

https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html


https://github.com/aws/sagemaker-inference-toolkit


## （おまけ）カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する

In [24]:
import boto3
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-1")

In [25]:
container

'354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.2-1'

In [30]:
est_lightgbm5 = Estimator(
    #image,
    container, # xgboostのbuilt-inコンテナ
    role,
    instance_count=1,
    #instance_type="ml.m4.2xlarge",
    instance_type="local",
    hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0},
    source_dir='./src_builtin_container',
    entry_point='train_practice.py'
    #entry_point='./practice_src/train_practice.sh'
    )

In [31]:
est_lightgbm5.fit({'train':train_s3, 'validation': valid_s3}, logs=True)

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Creating i5fghyfypp-algo-1-yfcz5 ... 
Creating i5fghyfypp-algo-1-yfcz5 ... done
Attaching to i5fghyfypp-algo-1-yfcz5
i5fghyfypp-algo-1-yfcz5 | [2022-10-04 04:28:16.976 6a1e6e45f30e:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
i5fghyfypp-algo-1-yfcz5 | INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
i5fghyfypp-algo-1-yfcz5 | INFO:sagemaker-containers:Failed to parse hyperparameter boosting_type value gbdt to Json.
i5fghyfypp-algo-1-yfcz5 | Returning the value itself
i5fghyfypp-algo-1-yfcz5 | INFO:sagemaker-containers:Failed to parse hyperparameter objective value regression to Json.
i5fghyfypp-algo-1-yfcz5 | Returning the value itself
i5fghyfypp-algo-1-yfcz5 | INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
i5fghyfypp-algo-1-yfcz5 | INFO:sagemaker_xgboost_container.training:Invoking user training script.
i5fghyfypp-algo-1-yfcz5 | INFO:sagemaker-containers:Module train_practice does not provide a setup.

このコンテナでservingをするにはどうすればいいか

In [42]:
predictor5 = est_lightgbm5.deploy(1, 'local', serializer=csv_serializer) 

!f8onwze1xo-algo-1-ixw9g | 172.18.0.1 - - [04/Oct/2022:05:49:58 +0000] "GET /ping HTTP/1.1" 200 1 "-" "python-urllib3/1.26.8"


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 854, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 916, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 859, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp95mqnqdk/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1


In [46]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor5.predict(payload).decode('utf-8')
print(predicted)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


f8onwze1xo-algo-1-ixw9g | Invoked with 102 records
19.95642073217597
27.844891841022335
23.747437427003455
21.961517177305176
33.70952263893306
16.546899933876215
20.7577247308279
21.58941351302627
28.44096446328559
21.573610198594977
16.520022349295115
18.56239893242527
33.70952263893306
21.66404760045202
18.839854556333133
20.524517944865078
23.512192914502315
19.720552829648888
14.831841119971708
25.48273874904075
24.232639474441545
21.624005932843115
24.961489794296718
31.737194191676068
21.634052928440624
28.40721160777621
21.408363849719503
14.831841119971708
22.218594550645975
21.174456098551236
21.78791955089051
14.831841119971708
29.996695633096042
22.44097524661187
33.83316205414468
26.41403196992683
33.70952263893306
17.366188662166092
27.56686070285819
30.785697489113854
19.36938873496206
20.70626548555591
17.759853567831996
27.888269821752413
20.521395163186774
14.831841119971708
24.776417537973362
24.965857100129327
19.649289821764185
21.026797620813866
33.70952263893306


In [48]:
predictor6 = est_lightgbm5.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=False) 

f8onwze1xo-algo-1-ixw9g | 172.18.0.1 - - [04/Oct/2022:05:53:02 +0000] "GET /ping HTTP/1.1" 200 1 "-" "python-urllib3/1.26.8"


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 854, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 916, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/local/image.py", line 859, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpf7485uip/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1


In [52]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor6.predict(payload).decode('utf-8')
print(predicted)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


19.95642073217597
27.844891841022335
23.747437427003455
21.961517177305176
33.70952263893306
16.546899933876215
20.7577247308279
21.58941351302627
28.44096446328559
21.573610198594977
16.520022349295115
18.56239893242527
33.70952263893306
21.66404760045202
18.839854556333133
20.524517944865078
23.512192914502315
19.720552829648888
14.831841119971708
25.48273874904075
24.232639474441545
21.624005932843115
24.961489794296718
31.737194191676068
21.634052928440624
28.40721160777621
21.408363849719503
14.831841119971708
22.218594550645975
21.174456098551236
21.78791955089051
14.831841119971708
29.996695633096042
22.44097524661187
33.83316205414468
26.41403196992683
33.70952263893306
17.366188662166092
27.56686070285819
30.785697489113854
19.36938873496206
20.70626548555591
17.759853567831996
27.888269821752413
20.521395163186774
14.831841119971708
24.776417537973362
24.965857100129327
19.649289821764185
21.026797620813866
33.70952263893306
22.770867837558004
25.12436361101226
32.04499227317

# End Of Containts ================